In [1]:
%matplotlib inline

In [2]:
import torch
import os
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import finalprojectneuralnetworks as fpnn
import csv

In [22]:
import random


with open('Dummy_Test.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    col_labels = ["AF3","F7","F3","FC5","T7","P7","O1","O2","P8","T8","FC6","F4","F8","AF4","eyeDetection"]
    spamwriter.writerow(col_labels)
    list_a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,0]
    list_b = [21,22,23,24,25,26,27,28,29,30,31,32,33,34,1]
    for i in range(500):
        random_number = random.randint(1, 10)
        if random_number > 5:  
            spamwriter.writerow(list_a)
        else:
            spamwriter.writerow(list_b)

In [3]:
# root_directory = 'D:\College\CS_545\Final_Project'
# fp_csv_file = 'EEG_Eye_State.csv'
# train_fraction = 0.8
# validate_fraction = 0.1
# X = pd.read_csv(os.path.join(root_directory,fp_csv_file), delimiter=',', usecols=range(15)).dropna(axis=0).to_numpy()
# n_samples = X.shape[0]
# rows = np.arange(n_samples)
# np.random.shuffle(rows)

# n_train = round(n_samples * train_fraction)
# n_validate = round(n_samples * validate_fraction)

# col_labels = ["AF3","F7","F3","FC5","T7","P7","O1","O2","P8","T8","FC6","F4","F8","AF4","eyeDetection"]
# Xtrain = X[rows[:n_train], :]
# Xvalidate = X[rows[n_train:n_train + n_validate], :]
# Xtest = X[rows[n_train + n_validate:], :]
# Xtrain = pd.DataFrame(Xtrain, columns = col_labels)
# Xvalidate = pd.DataFrame(Xvalidate, columns = col_labels)
# Xtest = pd.DataFrame(Xtest, columns = col_labels)
# Xtrain = Xtrain.astype({"eyeDetection": int})
# Xvalidate = Xvalidate.astype({"eyeDetection": int})
# Xtest = Xtest.astype({"eyeDetection": int})
# print(Xtrain.iloc[0])
# print( type(X))
# Xtrain.to_csv("EEG_Eye_State_Train.csv", encoding='utf-8', index_label=False, index=False, quoting=csv.QUOTE_NONNUMERIC )
# Xvalidate.to_csv("EEG_Eye_State_Validate.csv", encoding='utf-8', index_label=False, index=False, quoting=csv.QUOTE_NONNUMERIC )
# Xtest.to_csv("EEG_Eye_State_Test.csv", encoding='utf-8', index_label=False, index=False, quoting=csv.QUOTE_NONNUMERIC )

In [5]:


# root_directory = 'D:\College\CS_545\Final_Project'
root_directory = '/s/chopin/l/grad/acf003/CS_545'
fp_csv_file = 'EEG_Eye_State_Train.csv'
eeg_data = pd.read_csv(os.path.join(root_directory,fp_csv_file), delimiter=',', usecols=range(15)).dropna(axis=0)

print(eeg_data.iloc[0, :])
open_or_closed = eeg_data.iloc[0, 14]
channel_data = eeg_data.iloc[0, 0:14]
channel_data = np.array([channel_data])
channel_data = channel_data.astype('float').reshape(-1, 14)
print(open_or_closed)
print(channel_data)
eeg_data.shape

AF3             4309.74
F7              4052.82
F3              4285.13
FC5             4155.38
T7              4369.23
P7              4646.67
O1              4100.00
O2              4649.74
P8              4221.54
T8              4230.26
FC6             4206.15
F4              4292.82
F8              4598.97
AF4             4362.56
eyeDetection       0.00
Name: 0, dtype: float64
0
[[4309.74 4052.82 4285.13 4155.38 4369.23 4646.67 4100.   4649.74 4221.54
  4230.26 4206.15 4292.82 4598.97 4362.56]]


(11984, 15)

In [6]:
# root_directory = 'D:\College\CS_545\Final_Project'
root_directory = '/s/chopin/l/grad/acf003/CS_545'

train_file = 'EEG_Eye_State_Train.csv'
validate_file = 'EEG_Eye_State_Validate.csv'
test_file = 'EEG_Eye_State_Test.csv'

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
print(train_dataset.__len__())
train_dataset.__getitem__(0)
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6             float64
F4              float64
F8              float64
AF4             float64
eyeDetection      int64
dtype: object
AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6             float64
F4              float64
F8              float64
AF4             float64
eyeDetection      int64
dtype: object
AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6         

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [8]:
model = fpnn.NeuralNetwork(28*28, [512, 512], 10).to(device)
print(model)

i == 0
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (Linear0): Linear(in_features=784, out_features=512, bias=True)
    (ReLU0): ReLU()
    (Linear1): Linear(in_features=512, out_features=512, bias=True)
    (ReLU1): ReLU()
    (Linear2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[ 0.0102, -0.0323,  0.0691,  0.0265, -0.0896, -0.0404, -0.0841, -0.0201,
          0.0491, -0.0167]], device='cuda:0', grad_fn=<AddmmBackward>)
Predicted class: tensor([2], device='cuda:0')


In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    # first = True
    for batch, (X, T) in enumerate(dataloader):
        X, T = X.to(device), T.to(device)
      
        # Compute prediction error
        Y = model(X.float())
        # if first:
        #     first = False
        #     print('Y.shape', Y.shape)
        #     print('T.shape', T.shape)
        #     print('T: ', T[0])
        loss = loss_fn(Y, T.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         if batch % 50 == 0:
#             loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
           

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, T in dataloader:
            X, T = X.to(device), T.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, T.long()).item()
            correct += (pred.argmax(1) == T).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
def train_for_epochs(epochs, train_dataloader, model, loss_fn, optimizer):
    for t in range(epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        if t % 10 == 0:
            print(f"Epoch {t}\n-------------------------------")
            test(test_dataloader, model, loss_fn)
    print("Done!")

In [13]:
# fashion_training_data = datasets.FashionMNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
# )
# fashion_train_dataloader = DataLoader(fashion_training_data, batch_size=64)
# n_samples, n_inputs = train_dataset.__len__(), 28*28
# n_outputs = 1
# n_hiddens = [10, 10]
# model = fpnn.NeuralNetwork(28*28, n_hiddens, 10).to(device)
# print(model)

# learning_rate = 0.5 / (n_samples * n_outputs)  ## Larger learning rate

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CrossEntropyLoss()

# train(fashion_train_dataloader, model, loss_fn, optimizer)

In [14]:
for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 14])
Shape of y:  torch.Size([64]) torch.float64


In [18]:
n_samples, n_inputs = train_dataset.__len__(), 14
n_outputs = 2
n_hiddens = [500, 500]
model = fpnn.NeuralNetwork(n_inputs, n_hiddens, n_outputs).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 50
train_for_epochs(epochs, train_dataloader, model, loss_fn, optimizer)

i == 0
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (Linear0): Linear(in_features=14, out_features=500, bias=True)
    (ReLU0): ReLU()
    (Linear1): Linear(in_features=500, out_features=500, bias=True)
    (ReLU1): ReLU()
    (Linear2): Linear(in_features=500, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 53.2%, Avg loss: 12.534327 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 53.3%, Avg loss: 1.153025 

Epoch 20
-------------------------------
Test Error: 
 Accuracy: 57.3%, Avg loss: 0.677882 

Epoch 30
-------------------------------
Test Error: 
 Accuracy: 53.3%, Avg loss: 0.690054 

Epoch 40
-------------------------------
Test Error: 
 Accuracy: 53.3%, Avg loss: 0.691178 

Done!
